## Mutation is bad  - `Sort` method in List

Here Sort mutates the list while we trying to find sum in another parallel task

In [1]:
using static System.Linq.Enumerable;
using static System.Console;

var nums = Range(-10000, 20001).Reverse().ToList();

Action task1 = () => WriteLine(nums.Sum());
Action task2 = () => {nums.Sort(); WriteLine(nums.Sum());};
Parallel.Invoke(task1, task2);

40624


0


## With `OrderBy` extension method

In [1]:
Action task3 = () => WriteLine(nums.OrderBy(x => x).Sum());
Parallel.Invoke(task1, task2, task3);

-623


0


0


## Task Monad
Makes tasks composable

In [1]:
public async static Task<U> Select<T, U>(this Task<T> task, Func<T,U> mapper)
{
    var result = await task;
    return mapper(result);
}

public async static Task<R> SelectMany<T,U, R>(this Task<T> task, Func<T, Task<U>> mapper, Func<T, U, R> selector)
{
    var result = await task;
    return await mapper(result).Select(x => selector(result, x));
}

var task1 = Task.Run(() => 10);
var task2 = Task.Run(() => 20);
var task3 = Task.Run(() => 30);

var result = from x in task1
                from y in task2
                from z in task3
                select x * y * z;

await result

6000